In [1]:
# 🔢 Cell 1 — Install MCP SDK and supporting libraries (logs visible)
%pip install mcp python-dotenv requests httpx uvicorn

Note: you may need to restart the kernel to use updated packages.


Docker image install failed. Need:
git clone https://github.com/modelcontextprotocol/python-sdk.git
pip install -e ".[sqlite]"

In [2]:
# 🔢 Cell 2 — Load .env and servers_config.json into a Configuration class

import os
import json
from dotenv import load_dotenv

class Configuration:
    """Manages .env loading and MCP server config."""
    
    def __init__(self, config_path="servers_config.json"):
        load_dotenv()
        self.api_key = os.getenv("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("⚠️ OPENAI_API_KEY not found in environment variables.")
        self.server_config = self.load_config(config_path)
    
    @staticmethod
    def load_config(path: str) -> dict:
        with open(path, "r") as f:
            return json.load(f)

# Instantiate the config
config = Configuration()
print("✅ Loaded server config for:", list(config.server_config["mcpServers"].keys()))

✅ Loaded server config for: ['sqlite']


In [ ]:
# cd to mcp-sdk-python/python-sdk


In [6]:
# 🔢 Cell 3 — Connect to MCP Server (HTTP mode) and list available tools

from mcp import ClientSession
from mcp.client.http import http_client

class MCPHttpServer:
    """Connects to a running HTTP-based MCP server."""

    def __init__(self, name: str, url: str):
        self.name = name
        self.url = url
        self.session = None

    async def connect(self):
        client = http_client(self.url)
        read, write = await client.__aenter__()
        self.session = await ClientSession(read, write).__aenter__()
        await self.session.initialize()
        print(f"✅ Connected to MCP HTTP server: {self.name}")

    async def list_tools(self):
        if not self.session:
            raise RuntimeError("Session not initialized.")
        tools_response = await self.session.list_tools()
        tools = [tool.name for (kind, items) in tools_response if kind == "tools" for tool in items]
        return tools

# Run connection
async def run_tool_discovery():
    sqlite_url = config.server_config["mcpServers"]["sqlite"]["url"]
    server = MCPHttpServer("sqlite", sqlite_url)
    await server.connect()
    tools = await server.list_tools()
    print("🔧 Available tools:", tools)

await run_tool_discovery()

✅ Connected to MCP server: sqlite


RuntimeError: Attempted to exit a cancel scope that isn't the current tasks's current cancel scope

In [ ]:
import mcp
print("✅ MCP SDK is now installed and usable!")

In [ ]:
# 🧪 TEMP — Check if MCP is importable
import importlib

spec = importlib.util.find_spec("mcp")
print("✅ MCP is importable!" if spec else "❌ MCP is NOT importable")

In [ ]:
# 🧪 TEMP — Inspect what's inside the mcp module
import mcp
print("📦 Available in mcp:", dir(mcp))

In [ ]:
# 🔢 Cell 4 — Register the score_ikigai tool using kernel's Python (reliable)

import subprocess
import sys

cmd = [
    sys.executable, "-m", "mcp.cli.tool", "add",
    "--name", "score_ikigai",
    "--description", "Scores two career paths using Ikigai dimensions",
    "--input", "tools/tool-ikigai/score_ikigai.schema.json",
    "--db-path", "./test.db"
]

result = subprocess.run(cmd, capture_output=True, text=True)

if result.returncode == 0:
    print("✅ Tool registered successfully!")
    print(result.stdout)
else:
    print("❌ Tool registration failed:")
    print(result.stderr)